In [452]:
import pandas as pd

In [453]:
df=pd.read_csv('data/car_prediction_data.csv')
df

,Car_Name,Year,Selling_Price,Present_Price,Kms_Driven,Fuel_Type,Seller_Type,Transmission,Owner
0,ritz,2014,3.35,5.59,27000,Petrol,Dealer,Manual,0
1,sx4,2013,4.75,9.54,43000,Diesel,Dealer,Manual,0
2,ciaz,2017,7.25,9.85,6900,Petrol,Dealer,Manual,0
3,wagon r,2011,2.85,4.15,5200,Petrol,Dealer,Manual,0
4,swift,2014,4.60,6.87,42450,Diesel,Dealer,Manual,0
...,...,...,...,...,...,...,...,...,...
296,city,2016,9.50,11.60,33988,Diesel,Dealer,Manual,0
297,brio,2015,4.00,5.90,60000,Petrol,Dealer,Manual,0
298,city,2009,3.35,11.00,87934,Petrol,Dealer,Manual,0
299,city,2017,11.50,12.50,9000,Diesel,Dealer,Manual,0


In [454]:
## dependent and independet feature
X = df.drop(labels=['Selling_Price'],axis=1)
Y = df[['Selling_Price']]

In [455]:
X

,Car_Name,Year,Present_Price,Kms_Driven,Fuel_Type,Seller_Type,Transmission,Owner
0,ritz,2014,5.59,27000,Petrol,Dealer,Manual,0
1,sx4,2013,9.54,43000,Diesel,Dealer,Manual,0
2,ciaz,2017,9.85,6900,Petrol,Dealer,Manual,0
3,wagon r,2011,4.15,5200,Petrol,Dealer,Manual,0
4,swift,2014,6.87,42450,Diesel,Dealer,Manual,0
...,...,...,...,...,...,...,...,...
296,city,2016,11.60,33988,Diesel,Dealer,Manual,0
297,brio,2015,5.90,60000,Petrol,Dealer,Manual,0
298,city,2009,11.00,87934,Petrol,Dealer,Manual,0
299,city,2017,12.50,9000,Diesel,Dealer,Manual,0


In [456]:
Y

,Selling_Price
0,3.35
1,4.75
2,7.25
3,2.85
4,4.60
...,...
296,9.50
297,4.00
298,3.35
299,11.50


In [457]:
# Segregating numerical and categorical variables
categorical_cols = X.select_dtypes(include='object').columns
numerical_cols = X.select_dtypes(exclude='object').columns

In [459]:
numerical_cols

Index(['Year', 'Present_Price', 'Kms_Driven', 'Owner'], dtype='object')

In [427]:
pip install category_encoders

Note: you may need to restart the kernel to use updated packages.


In [460]:
from sklearn.impute import SimpleImputer ## HAndling Missing Values
from sklearn.preprocessing import StandardScaler # HAndling Feature Scaling
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import TargetEncoder
import category_encoders as ce  
## pipelines
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [461]:
import category_encoders as ce

num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Define categorical pipelines for each categorical column
cat_pipeline_Car_Name = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('category', ce.TargetEncoder()),  # Using TargetEncoder for encoding categorical variables
    ('scaler', StandardScaler(with_mean=False))
])

cat_pipeline_Fuel_Type_Seller_Type_Transmission = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehotencoder', OneHotEncoder()),  # Using OneHotEncoder for encoding categorical variables
    ('scaler', StandardScaler(with_mean=False))
])

# Combine numerical and categorical pipelines using ColumnTransformer
preprocessor = ColumnTransformer([
    ('num_pipeline', num_pipeline, numerical_cols),
    ('cat_pipeline_Car_Name', cat_pipeline_Car_Name, ['Car_Name']),
    ('cat_pipeline_Fuel_Type_Seller_Type_Transmission', cat_pipeline_Fuel_Type_Seller_Type_Transmission, ['Fuel_Type', 'Seller_Type', 'Transmission'])
])


In [462]:

from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.30,random_state=30)

In [470]:
X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.20,random_state=30)

# Reset indexes to avoid mismatched index errors
X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)

X_train = pd.DataFrame(preprocessor.fit_transform(X_train, y_train.values.ravel()), columns=preprocessor.get_feature_names_out())
X_test = pd.DataFrame(preprocessor.transform(X_test), columns=preprocessor.get_feature_names_out())

In [464]:
X_train

,num_pipeline__Year,num_pipeline__Present_Price,num_pipeline__Kms_Driven,num_pipeline__Owner,cat_pipeline_Car_Name__0,cat_pipeline_Fuel_Type_Seller_Type_Transmission__Fuel_Type_CNG,cat_pipeline_Fuel_Type_Seller_Type_Transmission__Fuel_Type_Diesel,cat_pipeline_Fuel_Type_Seller_Type_Transmission__Fuel_Type_Petrol,cat_pipeline_Fuel_Type_Seller_Type_Transmission__Seller_Type_Dealer,cat_pipeline_Fuel_Type_Seller_Type_Transmission__Seller_Type_Individual,cat_pipeline_Fuel_Type_Seller_Type_Transmission__Transmission_Automatic,cat_pipeline_Fuel_Type_Seller_Type_Transmission__Transmission_Manual
0,-0.609359,-0.816218,0.914307,-0.185695,4.779932,0.0,0.000000,2.444529,0.000000,2.128064,0.000000,3.217494
1,0.468622,0.896577,0.314742,-0.185695,6.653065,0.0,2.462348,0.000000,2.128064,0.000000,0.000000,3.217494
2,0.468622,0.314851,-0.724347,-0.185695,6.222343,0.0,0.000000,2.444529,2.128064,0.000000,3.217494,0.000000
3,-0.609359,-0.062972,0.627797,-0.185695,5.402459,0.0,2.462348,0.000000,2.128064,0.000000,0.000000,3.217494
4,0.468622,-0.701072,-0.651518,-0.185695,4.571033,0.0,0.000000,2.444529,0.000000,2.128064,0.000000,3.217494
...,...,...,...,...,...,...,...,...,...,...,...,...
235,0.468622,-0.779035,-0.615103,-0.185695,4.850529,0.0,0.000000,2.444529,0.000000,2.128064,0.000000,3.217494
236,-0.609359,-0.780235,-0.578688,-0.185695,4.704028,0.0,0.000000,2.444529,0.000000,2.128064,0.000000,3.217494
237,-0.250032,0.310053,0.794175,-0.185695,7.246752,0.0,0.000000,2.444529,2.128064,0.000000,0.000000,3.217494
238,-0.250032,-0.733457,-0.105300,-0.185695,4.775879,0.0,0.000000,2.444529,0.000000,2.128064,0.000000,3.217494


In [465]:



from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet
from sklearn.tree import DecisionTreeRegressor
from  sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error


In [466]:
import numpy as np
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [467]:
## Train multiple models
## Model Evaluation
from sklearn.metrics import r2_score
from sklearn.tree import DecisionTreeRegressor

models = {
    'LinearRegression': LinearRegression(),
    'Lasso': Lasso(),
    'Ridge': Ridge(),
    'Elasticnet': ElasticNet(),
    'DecisionTree': DecisionTreeRegressor(),
    'RandomForest': RandomForestRegressor()
}

model_list = []
r2_list = []

for model_name, model in models.items():
    model.fit(X_train, y_train)
    # Make Predictions
    y_pred = model.predict(X_test)

    mae, rmse, r2_square = evaluate_model(y_test, y_pred)

    print(model_name)
    model_list.append(model_name)

    print('Model Training Performance')
    print("RMSE:", rmse)
    print("MAE:", mae)
    print("R2 score:", r2_square * 100)
    
    # R^2 score
    r2 = r2_score(y_test, y_pred)
    if model_name == 'DecisionTree':
        print('Decision Tree Regression R^2 score:', r2)
    else:
        print("R2 Score:", r2)

    print('=' * 35)
    print('\n')

    r2_list.append(r2)

    print('=' * 35)
    print('\n')


LinearRegression
Model Training Performance
RMSE: 2.716612172111316
MAE: 1.703754562628074
R2 score: 83.21811695374457
R2 Score: 0.8321811695374457




Lasso
Model Training Performance
RMSE: 3.8593153775874955
MAE: 2.488218249119388
R2 score: 66.13072147844794
R2 Score: 0.6613072147844794




Ridge
Model Training Performance
RMSE: 2.719577427524815
MAE: 1.7039863333475345
R2 score: 83.1814612038692
R2 Score: 0.8318146120386921




Elasticnet
Model Training Performance
RMSE: 3.796352385785084
MAE: 2.140270989861433
R2 score: 67.22683072061062
R2 Score: 0.6722683072061062




DecisionTree
Model Training Performance
RMSE: 4.687495398904846
MAE: 1.8934426229508199
R2 score: 50.03485932369824
Decision Tree Regression R^2 score: 0.5003485932369824






d:\desktop\Car_price_Prediction\venv\lib\site-packages\sklearn\base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForest
Model Training Performance
RMSE: 2.722939683082406
MAE: 1.6506950819672133
R2 score: 83.13984945883317
R2 Score: 0.8313984945883317






In [436]:
df=pd.read_csv('data/Cleaned_Car_data.csv')

In [437]:
df.drop(labels=['Unnamed: 0'],axis=1,inplace=True)

In [438]:
## dependent and independet feature
X = df.drop(labels=['Price'],axis=1)
Y = df[['Price']]

In [439]:
# Segregating numerical and categorical variables
categorical_cols = X.select_dtypes(include='object').columns
numerical_cols = X.select_dtypes(exclude='object').columns

In [440]:
from sklearn.impute import SimpleImputer ## HAndling Missing Values
from sklearn.preprocessing import StandardScaler # HAndling Feature Scaling
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import TargetEncoder
import category_encoders as ce  
## pipelines
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [441]:
import category_encoders as ce

num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Define categorical pipelines for each categorical column
cat_pipeline_name_company = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('category', OneHotEncoder()),  # Using TargetEncoder for encoding categorical variables
    ('scaler', StandardScaler(with_mean=False))
])

cat_pipeline_fuel_type = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehotencoder', OneHotEncoder(handle_unknown='ignore')),  # Using OneHotEncoder for encoding categorical variables
    ('scaler', StandardScaler(with_mean=False))
])

# Combine numerical and categorical pipelines using ColumnTransformer
preprocessor = ColumnTransformer([
    ('num_pipeline', num_pipeline, numerical_cols),
    ('cat_pipeline_name_company', cat_pipeline_name_company, ['name','company']),
    ('cat_pipeline_fuel_type', cat_pipeline_fuel_type, ['fuel_type'])
])


In [442]:

from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.30,random_state=30)

In [443]:
X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.20,random_state=30)

# Reset indexes to avoid mismatched index errors
X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)

X_train = pd.DataFrame(preprocessor.fit_transform(X_train, y_train.values.ravel()), columns=preprocessor.get_feature_names_out())
X_test = pd.DataFrame(preprocessor.transform(X_test), columns=preprocessor.get_feature_names_out())

ValueError: Shape of passed values is (652, 1), indices imply (652, 257)

In [444]:
X_train

,name,company,year,kms_driven,fuel_type
0,Maruti Suzuki Wagon,Maruti,2015,22000,Petrol
1,Ford EcoSport Titanium,Ford,2014,44000,Diesel
2,Hyundai Verna Transform,Hyundai,2008,45000,Petrol
3,Toyota Fortuner 3.0,Toyota,2015,120000,Diesel
4,Chevrolet Spark LS,Chevrolet,2010,41000,Petrol
...,...,...,...,...,...
647,Mahindra Scorpio,Mahindra,2015,22000,Diesel
648,Maruti Suzuki Ertiga,Maruti,2012,51000,Diesel
649,Toyota Corolla Altis,Toyota,2009,132000,Petrol
650,Hyundai Verna 1.6,Hyundai,2017,12000,Petrol


In [412]:


import numpy as np
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square


In [413]:



from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet
from sklearn.tree import DecisionTreeRegressor
from  sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error


In [414]:
## Train multiple models
## Model Evaluation
from sklearn.metrics import r2_score
from sklearn.tree import DecisionTreeRegressor

models = {
    'LinearRegression': LinearRegression(),
    'Lasso': Lasso(),
    'Ridge': Ridge(),
    'Elasticnet': ElasticNet(),
    'DecisionTree': DecisionTreeRegressor(),
    'RandomForest': RandomForestRegressor()
}

model_list = []
r2_list = []

for model_name, model in models.items():
    model.fit(X_train, y_train)
    # Make Predictions
    y_pred = model.predict(X_test)

    mae, rmse, r2_square = evaluate_model(y_test, y_pred)

    print(model_name)
    model_list.append(model_name)

    print('Model Training Performance')
    print("RMSE:", rmse)
    print("MAE:", mae)
    print("R2 score:", r2_square * 100)
    
    # R^2 score
    r2 = r2_score(y_test, y_pred)
    if model_name == 'DecisionTree':
        print('Decision Tree Regression R^2 score:', r2)
    else:
        print("R2 Score:", r2)

    print('=' * 35)
    print('\n')

    r2_list.append(r2)

    print('=' * 35)
    print('\n')


LinearRegression
Model Training Performance
RMSE: 2.716612172111316
MAE: 1.703754562628074
R2 score: 83.21811695374457
R2 Score: 0.8321811695374457




Lasso
Model Training Performance
RMSE: 3.8593153775874955
MAE: 2.488218249119388
R2 score: 66.13072147844794
R2 Score: 0.6613072147844794




Ridge
Model Training Performance
RMSE: 2.719577427524815
MAE: 1.7039863333475345
R2 score: 83.1814612038692
R2 Score: 0.8318146120386921




Elasticnet
Model Training Performance
RMSE: 3.796352385785084
MAE: 2.140270989861433
R2 score: 67.22683072061062
R2 Score: 0.6722683072061062




DecisionTree
Model Training Performance
RMSE: 2.749712950741479
MAE: 1.6436065573770495
R2 score: 82.80666505174823
Decision Tree Regression R^2 score: 0.8280666505174823






d:\desktop\Car_price_Prediction\venv\lib\site-packages\sklearn\base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForest
Model Training Performance
RMSE: 2.6207978478593614
MAE: 1.5913344262295082
R2 score: 84.3810277788118
R2 Score: 0.843810277788118






In [415]:
from sklearn.model_selection import GridSearchCV
lr=LinearRegression()

In [416]:
parms= {
    'fit_intercept': [True, False],
    'normalize': [True, False],
    'copy_X': [True, False],
    'n_jobs': [-1]
}

In [417]:
parms = {
    'fit_intercept': [True, False],
    'copy_X': [True, False],
    'n_jobs': [-1]
}
cv = GridSearchCV(lr, param_grid=parms, cv=5, scoring='r2')
cv.fit(X_train, y_train)


GridSearchCV(cv=5, estimator=LinearRegression(),
             param_grid={'copy_X': [True, False],
                         'fit_intercept': [True, False], 'n_jobs': [-1]},
             scoring='r2')

In [418]:
y_pred=cv.predict(X_test)

In [419]:
cv.best_score_

0.8914549787583692

In [420]:
r2_score(y_test,y_pred)

0.8322180067902321